# Задание 7

## 1. PropertyCreator (0.2 балла)

Напишите мета класс для создания свойств (property) класса из функций начинающихся с "set\_", "get\_" или "del_". Пример использования:
<code>
class TestPropertyCreator(metaclass=PropertyCreator):
    def \__init\__(self, lo):
        self.__x = None
        self.lo = lo

    def get_x(self):
        return self.__x

    def set_x(self, value):
        if value < self.lo:
            raise ValueError("Value must in condition: {} <= value".format(self.lo))
        self.__x = value
    
    def del_x(self):
        self.__x = "No more"

    pass


obj = TestPropertyCreator(5)
obj.x = 4
print(obj.x)
del (obj.x)
</code>

* Мета класс должен поддерживать наследование, в смысле создавать свойства у потомков.
* Должен поддерживать частичное описание свойств, т. е. например, описание одного метода get_val (без set_val и del_val).
* Поддерживать множественное использование одного свойства с одним именем в разных классах:

<code>
class A(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class A"

class B(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class B"

class C(metaclass=PropertyCreator):
    def set_x(self, value):
        self.value = "x in class C"
    def get_x(self):
        return self.value
</code>
* Должен уметь обрабатывать имен с несколькими подчеркиваниями "get_raw_text".

#### Решение

In [35]:
from collections import defaultdict, Counter

In [3]:
props = defaultdict(lambda: {'get': None, 'set': None, 'del': None})

In [4]:
props['x']

{'del': None, 'get': None, 'set': None}

In [6]:
'get_a'[4:]

'a'

In [11]:
'get_a'.startswith('get_')

True

In [ ]:
props.

In [22]:
class PropertyCreator(type):
    def __new__(cls, name, bases, attrs):
        props = defaultdict(lambda: {'get': None, 'set': None, 'del': None})
        res_attrs = {}
        for attr_name, attr_value in attrs.items():
            if attr_name.startswith('get_'):
                props[attr_name[4:]]['get'] = attr_value
            elif attr_name.startswith('set_'):
                props[attr_name[4:]]['set'] = attr_value
            elif attr_name.startswith('del_'):
                props[attr_name[4:]]['del'] = attr_value
            else:
                res_attrs[attr_name] = attr_value
        for prop_name, prop_funcs in props.items():
            res_attrs[prop_name] = property(prop_funcs['get'], prop_funcs['set'], prop_funcs['del'])
        return super(PropertyCreator, cls).__new__(cls, name, bases, res_attrs)

#### Протестируйте свое решение

In [25]:
def test_simple():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self, lo):
            self.__x = None
            self.lo = lo
            
        def get_x(self):
            return self.__x

        def set_x(self, value):
            if value < self.lo:
                raise ValueError("Value must in condition: {} <= value".format(self.lo))
            self.__x = value

        def del_x(self):
            self.__x = "No more"

    t = TestPropertyCreator(10)
    t.x = 20
    try:
        t.x = 5
    except ValueError as e:
        print(e)
    print(t.x)

In [26]:
test_simple()

Value must in condition: 10 <= value
20


In [28]:
def test_with_inheritance():
    class TestPropertyCreator(metaclass=PropertyCreator):
        pass

    class TestPropertyCreatorInheritance(TestPropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_x(self, value):
            self._secret_list.append("set")

    t = TestPropertyCreatorInheritance()
    t.x = 20
    try:
        t.x = 5
    except ValueError as e:
        print(e)
    print(t.x)
    print(t._secret_list)
    
test_with_inheritance()

0
['set', 'set', 'get']


In [30]:
def test_partially_defined():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_y(self, value):
            self._secret_list.append("set")
            self._y = value

    t = TestPropertyCreator()
    t.y = 20
    try:
        t.y = 5
    except ValueError as e:
        print(e)
    print(t.x)
    print(t._secret_list)

test_partially_defined()   

0
['set', 'set', 'get']


In [32]:
def test_sanity():
    class TestPropertyCreator(metaclass=PropertyCreator):
        _text = 0
        def get_raw_text(self):
             return 'raw_text'

        def get_text(self):
             return self._text % 2

        def set_text(self, value):
            try:
                self._text = int(value)
            except ValueError:
                raise TypeError("unproper value for text: {}".format(value))

    t = TestPropertyCreator()
    t.text = 20
    print(t.text)
    print(t.raw_text)

test_sanity()   

0
raw_text


In [33]:
def test_multiple_usages():
    class TestPropertyCreatorA(metaclass=PropertyCreator):
        def get_x(self):
            return 0
        
    class TestPropertyCreatorB(metaclass=PropertyCreator):
        def get_x(self):
            return 1
        
    class TestPropertyCreatorC(metaclass=PropertyCreator):
        def set_x(self, value):
            self.value = value + 1
        def get_x(self):
            return self.value

    t_A = TestPropertyCreatorA()
    t_B = TestPropertyCreatorB()
    t_C = TestPropertyCreatorC()
    t_C.x = 20
    print(t_A.x)
    print(t_B.x)
    print(t_C.x)

test_multiple_usages()   

0
1
21


## 2. InstanceCountExeptioner (0.2 балла)
Напишите метакласс InstanceCountExeptioner, который будет следить за количеством экземпляров класса, использующих его. Количество задается через поле класса \_\_max_instane\_count\_\_. Т. е. число экземпляров каждого класса регулируется отдельно. Если в классе не указано поле \_\_max_instane\_count\_\_, то используйте заранее заданное число в метаклассе (любое). Пример:

<code>
class TestInstanceCountExeptionerA(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 2
    def \__init\__(self, a):
        self.a = a


class TestInstanceCountExeptionerB(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 1
    def \__init\__(self, a):
        self.a = a

a_one = TestInstanceCountExeptionerA('one')
a_two = TestInstanceCountExeptionerA('two')
b_one = TestInstanceCountExeptionerB('one')
\# пока всё шло хорошо

\# а вот
a_three = TestInstanceCountExeptionerA('three')
\# выкенет исключение InstanceCountExeption (ваше собственное исключение)
</code>

#### Решение

In [153]:
# Опишите исключение InstanceCountExeption
class InstanceCountExeption(Exception):
    def __init__(self, *args, **kwargs):
        Exception.__init__(self, *args, **kwargs)

In [184]:
# Опишите мета класс InstanceCountExeptioner
class InstanceCountExeptioner(type):
    counter = Counter()
    max_instance_counts = defaultdict(int)
    
    def __new__(cls, name, bases, attrs):
        cls.max_instance_counts[name] = attrs.get('__max_instance_count__', 2)
        cls.counter[name] = 0
        return super().__new__(cls, name, bases, attrs)
    
    def __call__(cls, *args, **kwargs):
        name = cls.__name__
        if cls.counter[name] >= cls.max_instance_counts[name]:
            raise InstanceCountExeption("You can't create new object of this class")
        cls.counter[name] += 1
        return super().__call__(*args, **kwargs)

#### Протестируйте свое решение

In [179]:
import unittest

In [185]:
class Tester(unittest.TestCase):
    def setUp(self):
        class TestInstanceCountExeptionerA(metaclass=InstanceCountExeptioner):
            __max_instance_count__ = 2

            def __init__(self):
                self.a = 1

            def get(self):
                return self.a

        class TestInstanceCountExeptionerB(metaclass=InstanceCountExeptioner):
            __max_instance_count__ = 3

            def __init__(self):
                self.b = 2

            def get(self):
                return self.b
        
        self.A = TestInstanceCountExeptionerA
        self.B = TestInstanceCountExeptionerB

    
    def test_create(self):
        a1 = self.A()
        a2 = self.A()
        b = self.B()
        self.assertEqual(a1.get(), 1)
        self.assertEqual(b.get(), 2)
        
    def test_fail_create_a(self):
        a1 = self.A()
        a2 = self.A()
        with self.assertRaises(InstanceCountExeption) as e:
            a3 = self.A()

    def test_fail_create_b(self):
        b1 = self.B()
        b2 = self.B()
        b3 = self.B()
        with self.assertRaises(InstanceCountExeption) as e:
            b4 = self.B()

In [186]:
suite = unittest.TestLoader().loadTestsFromTestCase(Tester)
unittest.TextTestRunner(verbosity=2).run(suite)

test_create (__main__.Tester) ... ok
test_fail_create_a (__main__.Tester) ... ok
test_fail_create_b (__main__.Tester) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

## 3. JSONClassCreator (0.6 баллов)
Напишите метакласс, который будет по json представлению строить новый класс и обратно. Класс должен уметь следующее:
* Поддерживать сохранение и получение магических функций класса.
* Поддерживать сохранение и получение обычных функций.
* Поддерживать сохранение полей со стандартными типами.
* Уберите из сохранения следующие поля и методы: ['\_\_dict\_\_', '\_\_weakref\_\_', '\_\_module\_\_', '\_\_init\_\_']
* У создаваемого класса должна быть функция to_json_str

Формат json строки должен быть следующий:

<code>
{
    "name": название класса,
    "bases": базовые классы,
    "methods": методы класса,
    "attrs": поля класса
}
</code>

Рекомендации:
* Для получения кода функций используйте модуль <a href="http://python-lab.ru/documentation/27/stdlib/inspect.html">inspect</a>.
* Для того, чтобы запустить код функций, можно использовать exec.
* Можно не исправлять ошибку типа OSError: could not get source code - возникает для функций, полученных с помощью exec.

#### Пример использования

In [187]:
import json

In [2]:
class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

In [5]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

('name', 'Test')
('bases', ['ParentTest1', 'ParentTest2'])
('methods', {'f': 'def f(self, x):\n        print(x)', '__repr__': 'def __repr__(self):\n        return "Test(val={})".format(self.val)', '__str__': 'def __str__(self):\n        return "Test(val={})".format(self.val)'})
('attrs', {'__doc__': 'Тестовый класс', 'val': [1, 2, 3]})


In [6]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

In [7]:
tmp_obj = tmp()
tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

hi


(Test(val=[1, 2, 3]), None, [1, 2, 3], 'Тестовый класс')

In [10]:
tmp.__dict__

mappingproxy({'__doc__': 'Тестовый класс',
              '__module__': '__main__',
              '__repr__': <function __main__.__repr__>,
              '__str__': <function __main__.__str__>,
              'f': <function __main__.f>,
              'to_json_str': <function __main__.JSONClassCreator.to_json_str>,
              'val': [1, 2, 3]})

#### Решение

In [188]:
import inspect

In [282]:
class JSONClassCreator(type):
    def __new__(mcls, json_str):
        raw_class = json.loads(json_str)
        name = raw_class['name']
        bases = tuple(globals()[base] for base in raw_class['bases'])
        for method_source in raw_class['methods'].values():
            exec(method_source, globals())
        methods = {
            method_name: globals()[method_name] 
                   for method_name, method_source in raw_class['methods'].items()
        }
        attrs = raw_class['attrs']
        attrs.update(methods)
        return super().__new__(mcls, name, bases, attrs)

    def to_json_str(cls):
        exclude = ['__dict__', '__weakref__', '__module__', '__init__']
        
        name = cls.__name__
        bases = [base.__name__ for base in cls.__bases__]
        exclude.append('__class__')
        attrs = {
            name: val
            for name, val in inspect.getmembers(cls, lambda x: not inspect.isroutine(x)) if not name in exclude
        }
        methods = {
            name: inspect.getsource(val).strip()
            for name, val in cls.__dict__.items() if name not in exclude and name not in attrs.keys()
        }

        return json.dumps({
            "name": name,
            "bases": bases,
            "methods": methods,
            "attrs": attrs
        })

    pass

#### Проверьте свое решение на примере

In [283]:
class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

In [284]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

('name', 'Test')
('bases', ['ParentTest1', 'ParentTest2'])
('methods', {'f': 'def f(self, x):\n        print(x)', '__repr__': 'def __repr__(self):\n        return "Test(val={})".format(self.val)', '__str__': 'def __str__(self):\n        return "Test(val={})".format(self.val)'})
('attrs', {'__doc__': 'Тестовый класс', 'val': [1, 2, 3]})


In [285]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

In [286]:
tmp_obj = tmp()
tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

hi


(Test(val=[1, 2, 3]), None, [1, 2, 3], 'Тестовый класс')

In [287]:
tmp.__dict__

mappingproxy({'__doc__': 'Тестовый класс',
              '__module__': '__main__',
              '__repr__': <function __main__.__repr__>,
              '__str__': <function __main__.__str__>,
              'f': <function __main__.f>,
              'val': [1, 2, 3]})